In [ ]:
# 필수 패키지 설치 및 import
import subprocess
import sys

# 핵심 패키지들 설치
packages_to_install = {
    'torch': 'torch',
    'torchvision': 'torchvision', 
    'scikit-learn': 'sklearn',
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'joblib': 'joblib'
}

print("🔄 필수 패키지 설치 확인 중...")

for pip_name, import_name in packages_to_install.items():
    try:
        __import__(import_name)
        print(f"✅ {pip_name} 이미 설치됨")
    except ImportError:
        print(f"⚠️ {pip_name} 설치 중...")
        if pip_name == 'torch':
            subprocess.check_call([sys.executable, "-m", "pip", "install", "torch", "torchvision", "--user"])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name, "--user"])
        print(f"✅ {pip_name} 설치 완료")

# 필수 라이브러리 임포트
try:
    import pandas as pd
    import numpy as np
    import json
    import joblib
    from datetime import datetime, timedelta
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import mean_squared_error
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    import warnings
    warnings.filterwarnings('ignore')
    
    # PyTorch 관련
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import Dataset, DataLoader, random_split
    
    print("✅ 모든 라이브러리 임포트 성공!")
    
except ImportError as e:
    print(f"❌ 라이브러리 임포트 실패: {e}")
    print("📋 재실행 후 다시 시도해주세요.")
    raise

# 한글 폰트 설정 (시스템 폰트 자동 감지)
try:
    import matplotlib.font_manager as fm
    
    # macOS 시스템 한글 폰트 찾기
    system_fonts = [f.name for f in fm.fontManager.ttflist]
    korean_fonts = [f for f in system_fonts if any(keyword in f for keyword in ['Gothic', 'Malgun', 'NanumGothic', 'AppleGothic'])]
    
    if korean_fonts:
        plt.rcParams['font.family'] = korean_fonts[0]
        print(f"✅ 한글 폰트 설정: {korean_fonts[0]}")
    else:
        plt.rcParams['font.family'] = ['DejaVu Sans']
        print("⚠️ 한글 폰트를 찾을 수 없어 기본 폰트 사용")
        
    plt.rcParams['axes.unicode_minus'] = False
    
except Exception as e:
    print(f"⚠️ 폰트 설정 오류 (무시하고 진행): {e}")
    plt.rcParams['font.family'] = ['DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False

# GPU/CPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 디바이스: {device}")

print("\n" + "="*60)
print("📦 라이브러리 로드 완료!")
print(f"📅 현재 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python 버전: {sys.version.split()[0]}")
print(f"🔥 PyTorch 버전: {torch.__version__}")
print(f"📚 주요 패키지 버전:")
print(f"   - pandas: {pd.__version__}")
print(f"   - numpy: {np.__version__}")
print(f"   - sklearn: {sklearn.__version__}")
print("="*60)


In [ ]:
# 데이터 로드 및 전처리
print("🔄 데이터 로드 중...")

# 훈련 데이터 로드 (Task 3.5 결과)
train_path = '../../results/preprocessing/05_data_splitting/train_data_full.csv'
train_data = pd.read_csv(train_path)
train_data['date'] = pd.to_datetime(train_data['date'])

# 예측 템플릿 로드
pred_template_path = '../../results/preprocessing/05_data_splitting/prediction_template.csv'
pred_template = pd.read_csv(pred_template_path)
pred_template['date'] = pd.to_datetime(pred_template['date'])

# Lag 초기값 로드
lag_init_path = '../../results/preprocessing/05_data_splitting/lag_initialization.json'
with open(lag_init_path, 'r') as f:
    lag_init = json.load(f)

# CV 폴드 정보 로드
cv_metadata_path = '../../results/preprocessing/05_data_splitting/cv_folds_metadata.json'
with open(cv_metadata_path, 'r') as f:
    cv_metadata = json.load(f)

print(f"✅ 훈련 데이터: {train_data.shape} (기간: {train_data['date'].min()} ~ {train_data['date'].max()})")
print(f"✅ 예측 템플릿: {pred_template.shape} (기간: {pred_template['date'].min()} ~ {pred_template['date'].max()})")
print(f"✅ CV 설정: {cv_metadata['metadata']['total_folds']}개 폴드")

# LSTM 입력을 위한 피처 선택
feature_columns = [col for col in train_data.columns if col not in ['date', '최대전력(MW)']]
target_column = '최대전력(MW)'

print(f"✅ 선택된 피처 수: {len(feature_columns)}")
print(f"✅ 타겟 변수: {target_column}")

# 데이터 정렬 (날짜순)
train_data = train_data.sort_values('date').reset_index(drop=True)

print(f"\n📊 타겟 변수 통계:")
print(f"   평균: {train_data[target_column].mean():.0f} MW")
print(f"   표준편차: {train_data[target_column].std():.0f} MW")
print(f"   범위: {train_data[target_column].min():.0f} ~ {train_data[target_column].max():.0f} MW")

# 데이터 전처리를 위한 스케일러 설정
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# 훈련 데이터로 스케일러 학습
features_scaled = feature_scaler.fit_transform(train_data[feature_columns])
target_scaled = target_scaler.fit_transform(train_data[[target_column]])

print("✅ 데이터 스케일링 완료 (MinMaxScaler)")
print(f"   피처 스케일 범위: [0, 1]")
print(f"   타겟 스케일 범위: [0, 1]")


In [ ]:
# Custom Dataset 클래스 정의
class TimeSeriesDataset(Dataset):
    """시계열 데이터를 위한 Custom Dataset"""
    
    def __init__(self, features, targets, sequence_length=60):
        """
        Args:
            features: 스케일된 피처 데이터 (numpy array)
            targets: 스케일된 타겟 데이터 (numpy array)
            sequence_length: 입력 시퀀스 길이
        """
        self.features = features
        self.targets = targets.flatten()  # (n, 1) -> (n,)
        self.sequence_length = sequence_length
        
    def __len__(self):
        return len(self.features) - self.sequence_length
    
    def __getitem__(self, idx):
        # idx부터 sequence_length만큼의 피처 시퀀스
        feature_seq = self.features[idx:idx+self.sequence_length]
        
        # idx+sequence_length 시점의 타겟값
        target = self.targets[idx+self.sequence_length]
        
        return torch.FloatTensor(feature_seq), torch.FloatTensor([target])

# LSTM 모델 클래스 정의
class LSTMModel(nn.Module):
    """전력 수요 예측을 위한 LSTM 모델"""
    
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.2):
        """
        Args:
            input_size: 입력 피처 수
            hidden_size: LSTM 은닉층 크기
            num_layers: LSTM 레이어 수
            dropout: 드롭아웃 비율
        """
        super(LSTMModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM 레이어
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # 출력 레이어
        self.fc = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # LSTM 출력
        lstm_out, (h_n, c_n) = self.lstm(x)
        
        # 마지막 시점의 출력 사용
        last_output = lstm_out[:, -1, :]
        
        # 드롭아웃 적용
        dropped = self.dropout(last_output)
        
        # 선형 변환으로 최종 출력
        output = self.fc(dropped)
        
        return output

# 모델 하이퍼파라미터 설정
SEQUENCE_LENGTH = 60  # 60일 시퀀스로 다음날 예측
HIDDEN_SIZE = 128
NUM_LAYERS = 2
DROPOUT = 0.2
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 100
PATIENCE = 10  # Early stopping patience

print("🏗️ 모델 아키텍처 정의 완료!")
print(f"📐 하이퍼파라미터:")
print(f"   - Sequence Length: {SEQUENCE_LENGTH}")
print(f"   - Hidden Size: {HIDDEN_SIZE}")
print(f"   - Num Layers: {NUM_LAYERS}")
print(f"   - Dropout: {DROPOUT}")
print(f"   - Batch Size: {BATCH_SIZE}")
print(f"   - Learning Rate: {LEARNING_RATE}")
print(f"   - Max Epochs: {NUM_EPOCHS}")
print(f"   - Early Stopping Patience: {PATIENCE}")


In [ ]:
# 훈련 및 검증 함수 정의
def train_model(model, train_loader, val_loader, num_epochs, patience=10):
    """LSTM 모델 훈련"""
    
    # 손실 함수와 옵티마이저
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    # 훈련 기록
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    print(f"🚀 모델 훈련 시작 (디바이스: {device})")
    print("-" * 60)
    
    for epoch in range(num_epochs):
        # 훈련 모드
        model.train()
        train_loss = 0.0
        
        for batch_features, batch_targets in train_loader:
            batch_features = batch_features.to(device)
            batch_targets = batch_targets.to(device)
            
            # 순전파
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_targets)
            
            # 역전파
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # 검증 모드
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for batch_features, batch_targets in val_loader:
                batch_features = batch_features.to(device)
                batch_targets = batch_targets.to(device)
                
                outputs = model(batch_features)
                loss = criterion(outputs, batch_targets)
                val_loss += loss.item()
        
        # 평균 손실 계산
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        
        # 학습률 스케줄러 업데이트
        scheduler.step(avg_val_loss)
        
        # Early stopping 체크
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        # 진행상황 출력 (10 epoch마다)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1:3d}/{num_epochs}] | "
                  f"Train Loss: {avg_train_loss:.6f} | "
                  f"Val Loss: {avg_val_loss:.6f} | "
                  f"LR: {optimizer.param_groups[0]['lr']:.2e}")
        
        # Early stopping
        if patience_counter >= patience:
            print(f"\n⏹️ Early stopping at epoch {epoch+1}")
            print(f"   Best validation loss: {best_val_loss:.6f}")
            break
    
    # 최고 모델 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"✅ 최고 성능 모델 복원 (Val Loss: {best_val_loss:.6f})")
    
    return train_losses, val_losses

def calculate_rmse(y_true, y_pred):
    """RMSE 계산"""
    return np.sqrt(mean_squared_error(y_true, y_pred))

print("🔧 훈련 함수 정의 완료!")


In [ ]:
# 시계열 교차검증으로 모델 훈련
print("🔄 시계열 교차검증 시작...")

# 결과 저장용
cv_results = []
models_dict = {}

# 각 폴드별 훈련
for fold_id, fold_info in cv_metadata['folds'].items():
    print(f"\n{'='*60}")
    print(f"🔄 {fold_id} 훈련 시작")
    print(f"   훈련: {fold_info['train_size']}일, 검증: {fold_info['val_size']}일")
    print(f"   훈련 기간: {fold_info['train_start']} ~ {fold_info['train_end']}")
    print(f"   검증 기간: {fold_info['val_start']} ~ {fold_info['val_end']}")
    
    # 폴드별 데이터 분할
    train_start_idx = 0
    train_end_idx = fold_info['train_size']
    val_start_idx = train_end_idx
    val_end_idx = val_start_idx + fold_info['val_size']
    
    # 훈련/검증 데이터 준비
    train_features = features_scaled[:train_end_idx]
    train_targets = target_scaled[:train_end_idx]
    val_features = features_scaled[val_start_idx:val_end_idx]
    val_targets = target_scaled[val_start_idx:val_end_idx]
    
    # Dataset 및 DataLoader 생성
    train_dataset = TimeSeriesDataset(train_features, train_targets, SEQUENCE_LENGTH)
    val_dataset = TimeSeriesDataset(val_features, val_targets, SEQUENCE_LENGTH)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    print(f"   데이터셋 크기: 훈련 {len(train_dataset)}, 검증 {len(val_dataset)}")
    
    # 모델 초기화
    input_size = len(feature_columns)
    model = LSTMModel(
        input_size=input_size,
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
        dropout=DROPOUT
    ).to(device)
    
    print(f"   모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")
    
    # 모델 훈련
    train_losses, val_losses = train_model(
        model, train_loader, val_loader, NUM_EPOCHS, PATIENCE
    )
    
    # 검증 데이터로 RMSE 계산
    model.eval()
    val_predictions = []
    val_actuals = []
    
    with torch.no_grad():
        for batch_features, batch_targets in val_loader:
            batch_features = batch_features.to(device)
            outputs = model(batch_features)
            
            # CPU로 이동 및 스케일 복원
            pred_scaled = outputs.cpu().numpy()
            target_scaled_batch = batch_targets.cpu().numpy()
            
            pred_original = target_scaler.inverse_transform(pred_scaled)
            target_original = target_scaler.inverse_transform(target_scaled_batch)
            
            val_predictions.extend(pred_original.flatten())
            val_actuals.extend(target_original.flatten())
    
    # RMSE 계산
    fold_rmse = calculate_rmse(val_actuals, val_predictions)
    
    # 결과 저장
    fold_result = {
        'fold': fold_id,
        'train_size': fold_info['train_size'],
        'val_size': fold_info['val_size'],
        'val_rmse': fold_rmse,
        'final_train_loss': train_losses[-1],
        'final_val_loss': val_losses[-1],
        'epochs_trained': len(train_losses)
    }
    cv_results.append(fold_result)
    models_dict[fold_id] = model.state_dict().copy()
    
    print(f"✅ {fold_id} 완료!")
    print(f"   검증 RMSE: {fold_rmse:.2f} MW")
    print(f"   훈련 완료 에포크: {len(train_losses)}")

# 교차검증 결과 요약
print(f"\n{'='*60}")
print("📊 교차검증 결과 요약")
print(f"{'='*60}")

cv_rmses = [result['val_rmse'] for result in cv_results]
print(f"평균 RMSE: {np.mean(cv_rmses):.2f} ± {np.std(cv_rmses):.2f} MW")
print(f"최고 RMSE: {min(cv_rmses):.2f} MW")
print(f"최악 RMSE: {max(cv_rmses):.2f} MW")

print(f"\n폴드별 상세 결과:")
for result in cv_results:
    print(f"  {result['fold']}: {result['val_rmse']:.2f} MW "
          f"({result['epochs_trained']} epochs)")

print("✅ 교차검증 완료!")


In [ ]:
# 최종 모델 훈련 (전체 데이터)
print("🎯 최종 모델 훈련 (전체 훈련 데이터 사용)")

# 전체 훈련 데이터로 Dataset 생성
full_dataset = TimeSeriesDataset(features_scaled, target_scaled, SEQUENCE_LENGTH)

# 80-20 분할 (훈련-검증)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"전체 데이터셋: {len(full_dataset)} samples")
print(f"훈련: {len(train_dataset)}, 검증: {len(val_dataset)}")

# 최종 모델 초기화
final_model = LSTMModel(
    input_size=len(feature_columns),
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT
).to(device)

print(f"최종 모델 파라미터 수: {sum(p.numel() for p in final_model.parameters()):,}")

# 최종 모델 훈련
print("\n🚀 최종 모델 훈련 시작...")
final_train_losses, final_val_losses = train_model(
    final_model, train_loader, val_loader, NUM_EPOCHS, PATIENCE
)

print("✅ 최종 모델 훈련 완료!")


In [ ]:
# 527일 예측 생성
print("🔮 527일 예측 생성 중...")

def generate_future_predictions(model, last_sequence, pred_template, feature_scaler, target_scaler, sequence_length):
    """527일 미래 예측 생성"""
    model.eval()
    predictions = []
    
    # 초기 시퀀스 설정 (마지막 sequence_length일의 피처)
    current_sequence = last_sequence.copy()
    
    with torch.no_grad():
        for day_idx in range(len(pred_template)):
            # 현재 시퀀스를 텐서로 변환
            seq_tensor = torch.FloatTensor(current_sequence).unsqueeze(0).to(device)
            
            # 예측
            pred_scaled = model(seq_tensor).cpu().numpy()
            pred_original = target_scaler.inverse_transform(pred_scaled)[0, 0]
            predictions.append(pred_original)
            
            # 다음 날의 피처 가져오기 (예측 템플릿에서)
            if day_idx < len(pred_template) - 1:
                next_day_features = pred_template.iloc[day_idx][feature_columns].values
                next_day_features_scaled = feature_scaler.transform([next_day_features])[0]
                
                # 시퀀스 업데이트 (가장 오래된 것 제거, 새로운 것 추가)
                current_sequence = np.vstack([current_sequence[1:], next_day_features_scaled])
    
    return np.array(predictions)

# 마지막 sequence_length일의 피처 추출
last_sequence = features_scaled[-SEQUENCE_LENGTH:]

# 527일 예측 생성
future_predictions = generate_future_predictions(
    final_model, last_sequence, pred_template, 
    feature_scaler, target_scaler, SEQUENCE_LENGTH
)

print(f"✅ 527일 예측 완료!")
print(f"   예측 범위: {future_predictions.min():.0f} ~ {future_predictions.max():.0f} MW")
print(f"   예측 평균: {future_predictions.mean():.0f} MW")

# 대회 형식 날짜 변환 함수
def format_competition_date(date):
    """YYYY.M.D 형식으로 날짜 변환 (한 자리 숫자일 때 0 생략)"""
    return f"{date.year}.{date.month}.{date.day}"

# 제출 파일 생성
submission = pd.DataFrame({
    'date': [format_competition_date(date) for date in pred_template['date']],
    '최대전력(MW)': future_predictions
})

# 제출 파일 저장
submission_path = 'submission_lstm.csv'
submission.to_csv(submission_path, index=False)
print(f"✅ 제출 파일 저장: {submission_path}")

# 제출 파일 미리보기
print(f"\n📋 제출 파일 미리보기:")
print(submission.head(10))
print("...")
print(submission.tail(5))


In [ ]:
# 결과 시각화 및 모델 저장
print("📊 결과 시각화 및 모델 저장...")

# 1. 훈련 손실 시각화
plt.figure(figsize=(15, 10))

# 첫 번째 서브플롯: 최종 모델 훈련 손실
plt.subplot(2, 3, 1)
plt.plot(final_train_losses, label='Training Loss', color='blue')
plt.plot(final_val_losses, label='Validation Loss', color='red')
plt.title('Final Model Training History')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)

# 두 번째 서브플롯: CV RMSE 분포
plt.subplot(2, 3, 2)
cv_rmses = [result['val_rmse'] for result in cv_results]
plt.bar(range(len(cv_rmses)), cv_rmses, color='skyblue', edgecolor='navy')
plt.title('Cross-Validation RMSE by Fold')
plt.xlabel('Fold')
plt.ylabel('RMSE (MW)')
plt.xticks(range(len(cv_rmses)), [f'Fold {i+1}' for i in range(len(cv_rmses))])
for i, rmse in enumerate(cv_rmses):
    plt.text(i, rmse + 50, f'{rmse:.0f}', ha='center', va='bottom')
plt.grid(True, alpha=0.3)

# 세 번째 서브플롯: 예측값 분포
plt.subplot(2, 3, 3)
plt.hist(future_predictions, bins=30, alpha=0.7, color='green', edgecolor='black')
plt.title('Prediction Distribution')
plt.xlabel('Power Demand (MW)')
plt.ylabel('Frequency')
plt.axvline(future_predictions.mean(), color='red', linestyle='--', 
           label=f'Mean: {future_predictions.mean():.0f} MW')
plt.legend()
plt.grid(True, alpha=0.3)

# 네 번째 서브플롯: 월별 예측 패턴
plt.subplot(2, 3, 4)
pred_dates = pred_template['date']
monthly_avg = []
months = []
for month in range(1, 13):
    month_mask = pred_dates.dt.month == month
    if month_mask.any():
        monthly_avg.append(future_predictions[month_mask].mean())
        months.append(month)

plt.plot(months, monthly_avg, marker='o', linewidth=2, markersize=6)
plt.title('Monthly Average Predictions')
plt.xlabel('Month')
plt.ylabel('Average Power Demand (MW)')
plt.xticks(months)
plt.grid(True)

# 다섯 번째 서브플롯: 시계열 예측 그래프 (첫 90일)
plt.subplot(2, 3, 5)
first_90_days = pred_dates[:90]
first_90_predictions = future_predictions[:90]
plt.plot(first_90_days, first_90_predictions, linewidth=1.5, color='purple')
plt.title('First 90 Days Predictions')
plt.xlabel('Date')
plt.ylabel('Power Demand (MW)')
plt.xticks(rotation=45)
plt.grid(True)

# 여섯 번째 서브플롯: 모델 성능 요약
plt.subplot(2, 3, 6)
plt.axis('off')
summary_text = f'''
LSTM Model Performance Summary

Cross-Validation Results:
  Mean RMSE: {np.mean(cv_rmses):.1f} ± {np.std(cv_rmses):.1f} MW
  Best RMSE: {min(cv_rmses):.1f} MW
  Worst RMSE: {max(cv_rmses):.1f} MW

Model Architecture:
  Input Features: {len(feature_columns)}
  Hidden Size: {HIDDEN_SIZE}
  Num Layers: {NUM_LAYERS}
  Sequence Length: {SEQUENCE_LENGTH}
  Parameters: {sum(p.numel() for p in final_model.parameters()):,}

Prediction Stats:
  Mean: {future_predictions.mean():.0f} MW
  Std: {future_predictions.std():.0f} MW
  Range: {future_predictions.min():.0f} - {future_predictions.max():.0f} MW
'''
plt.text(0.05, 0.95, summary_text, fontsize=10, verticalalignment='top',
         fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='lightgray'))

plt.tight_layout()
plt.savefig('lstm_model_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# 모델 및 스케일러 저장
print("\n💾 모델 및 메타데이터 저장...")

# 모델 저장
torch.save({
    'model_state_dict': final_model.state_dict(),
    'model_architecture': {
        'input_size': len(feature_columns),
        'hidden_size': HIDDEN_SIZE,
        'num_layers': NUM_LAYERS,
        'dropout': DROPOUT
    },
    'training_config': {
        'sequence_length': SEQUENCE_LENGTH,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'num_epochs': NUM_EPOCHS,
        'patience': PATIENCE
    },
    'feature_columns': feature_columns,
    'target_column': target_column
}, 'lstm_model.pth')

# 스케일러 저장
joblib.dump(feature_scaler, 'feature_scaler_lstm.pkl')
joblib.dump(target_scaler, 'target_scaler_lstm.pkl')

# 결과 메타데이터 저장
metadata = {
    'model_type': 'LSTM',
    'creation_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'cv_results': cv_results,
    'final_model_performance': {
        'final_train_loss': final_train_losses[-1],
        'final_val_loss': final_val_losses[-1],
        'epochs_trained': len(final_train_losses)
    },
    'prediction_stats': {
        'mean': float(future_predictions.mean()),
        'std': float(future_predictions.std()),
        'min': float(future_predictions.min()),
        'max': float(future_predictions.max())
    },
    'hyperparameters': {
        'sequence_length': SEQUENCE_LENGTH,
        'hidden_size': HIDDEN_SIZE,
        'num_layers': NUM_LAYERS,
        'dropout': DROPOUT,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE
    }
}

with open('lstm_model_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print("✅ 저장 완료!")
print("   - lstm_model.pth: 모델 가중치")
print("   - feature_scaler_lstm.pkl: 피처 스케일러")
print("   - target_scaler_lstm.pkl: 타겟 스케일러")
print("   - lstm_model_metadata.json: 모델 메타데이터")
print("   - submission_lstm.csv: 대회 제출 파일")
print("   - lstm_model_analysis.png: 분석 시각화")

print(f"\n🎯 Task 5 완료!")
print(f"   평균 CV RMSE: {np.mean(cv_rmses):.1f} MW")
print(f"   제출 파일: submission_lstm.csv ({len(submission)} 예측)")
